In [1]:
import os, math, itertools
import numpy as np

In [2]:
from generate_test_sample import generate_sample
from sample_reader import SampleReader

In [3]:
samples_fv = SampleReader().read('/Users/andrewbzikadze/chihua/datasets/AbVitro/flu_time_course/FV/', ['25'])
samples_gmc = SampleReader().read('/Users/andrewbzikadze/chihua/datasets/AbVitro/flu_time_course/GMC/', ['8'])
samples_ido = SampleReader().read('/Users/andrewbzikadze/chihua/datasets/AbVitro/flu_time_course/IDO/')
samples_age = SampleReader().read('/Users/andrewbzikadze/chihua/datasets/age/')
samples_paired = SampleReader().read('/Users/andrewbzikadze/chihua/datasets/AbVitro/paired/')

In [4]:
from shm_kmer_model_estimator import ShmKmerModelEstimator

In [30]:
estimator = ShmKmerModelEstimator()
est_model = estimator.estimate_models_of_one_type((#samples_age,
                                                   #samples_ido,
                                                   #samples_fv,
                                                   samples_gmc,
                                                   samples_paired
                                                  ),
                                                  chains=['IGH', 'IGK', 'IGL'])

NoKNeighbours: IGH
NoKNeighbours: IGK
NoKNeighbours: IGL
Trivial: IGH
Trivial: IGK
Trivial: IGL


In [31]:
est_model['NoKNeighbours']['IGK']

,beta_shape1,beta_shape2,dir_shape1,dir_shape2,dir_shape3,success_optim_beta,success_optim_dir
AAAAA,14.883947,524.115840,9.524400,11.457888,2.202404,1.0,1.0
AAAAC,4.714259,89.543922,6.027751,19.057985,2.106033,1.0,1.0
AAAAG,12.362007,202.912625,13.623920,4.275669,2.316022,1.0,1.0
AAAAT,12.143571,134.778974,10.340163,18.148272,3.115601,1.0,1.0
AAACA,6.448083,99.447081,4.899089,13.440550,5.046643,1.0,1.0
AAACC,37.482542,554.526769,17.211704,135.785338,10.908592,1.0,1.0
AAACG,0.066978,25.416047,1.666667,0.666667,0.666667,0.0,0.0
AAACT,1.514663,26.494915,1.375733,1.634373,1.532660,1.0,1.0
AAAGA,39.502133,1028.528599,8.849149,16.730126,2.085852,1.0,1.0
AAAGC,15.152594,314.686507,24.164498,49.908382,10.420042,1.0,1.0


In [32]:
np.mean(est_model['NoKNeighbours']['IGH']['success_optim_beta'])

0.93359375

In [33]:
samples_paired['NoKNeighbours']['IGH'][:,6,:]

array([[21,  0,  0,  0]])

In [34]:
def apply_to_datasets(datasets, func):
    result = dict.fromkeys(datasets)
    for strategy in datasets:
        result[strategy] = dict.fromkeys(datasets[strategy])
        for chain_type in datasets[strategy]:
            result[strategy][chain_type] = \
            func(datasets[strategy][chain_type])
    return result

In [35]:
apply_to_datasets(est_model, (lambda x: x.isnull().any(axis=1).mean()))

{'NoKNeighbours': {'IGH': 0.0234375, 'IGK': 0.087890625, 'IGL': 0.0458984375},
 'Trivial': {'IGH': 0.009765625, 'IGK': 0.0791015625, 'IGL': 0.041015625}}

In [36]:
apply_to_datasets(est_model, (lambda x: x['success_optim_beta'].mean()))

{'NoKNeighbours': {'IGH': 0.93359375, 'IGK': 0.9462890625, 'IGL': 0.95703125},
 'Trivial': {'IGH': 0.98046875, 'IGK': 0.982421875, 'IGL': 0.98828125}}

In [37]:
apply_to_datasets(est_model, (lambda x: x['success_optim_dir'].mean()))

{'NoKNeighbours': {'IGH': 0.8974609375,
  'IGK': 0.9404296875,
  'IGL': 0.9501953125},
 'Trivial': {'IGH': 0.93359375, 'IGK': 0.951171875, 'IGL': 0.9619140625}}

In [13]:
test_sample = generate_sample(sample_size=1000,
                number_samples=20,
                real_beta_shape1=3,
                real_beta_shape2=15,
                real_dir_lambda=[10, 11, 12],
                nonmutated_ind=0)

print(test_sample)

np.mean(1 - test_sample[:,0] / np.sum(test_sample, axis=1, dtype=float))

[[981   7   7   5]
 [722 105  89  84]
 [831  33  40  96]
 [817  59  61  63]
 [838  43  52  67]
 [805  59  82  54]
 [892  35  28  45]
 [796  69  54  81]
 [842  38  57  63]
 [809  64  38  89]
 [774 100  26 100]
 [887  36  31  46]
 [768  69  94  69]
 [828  40  71  61]
 [913  23  21  43]
 [784  77  79  60]
 [919  24  25  32]
 [744  96  71  89]
 [739  80  47 134]
 [829  50  51  70]]


0.1741